# 🚀 **Módulo 6: Deploy como abrir um restaurante**

## **Aula 6.1: Exportando e salvando modelos**

---

### **Tá, mas o que é Deploy?**

Imagine que você é um chef que criou uma receita incrível. Agora você precisa:
- ✅ **Abrir um restaurante** (infraestrutura)
- ✅ **Contratar garçons** (interface)
- ✅ **Fazer propaganda** (divulgação)
- ✅ **Atender clientes** (produção)

**Deploy é a mesma coisa!** Vamos colocar nossa IA treinada para trabalhar no mundo real.

**Por que o Deploy é importante?**

É como abrir um negócio:
- **Sem deploy**: IA fica guardada no computador (como receita na gaveta)
- **Com deploy**: IA atende clientes reais (como restaurante funcionando)

---

**🖼️ Sugestão de imagem**: Chef abrindo restaurante vs deploy de modelo de IA

### **Setup Inicial - Preparando o Terreno**

In [ ]:
# Importações necessárias
import torch
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from transformers import AutoTokenizer, AutoModelForCausalLM
import gradio as gr
import streamlit as st
import json
import os
import warnings
warnings.filterwarnings('ignore')

print("✅ Bibliotecas importadas com sucesso!")

## **1. Preparando o modelo para deploy**

Vamos preparar nosso modelo treinado:

In [ ]:
# Carregando modelo treinado (simulado)
print("🤖 PREPARANDO MODELO PARA DEPLOY\n")

# Para demonstração, vamos usar o modelo base
model_name = "microsoft/DialoGPT-small"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)

print(f"✅ Modelo carregado: {model_name}")
print(f"📊 Parâmetros: {model.num_parameters():,}")

# Configurando tokenizer
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

# Função de inferência otimizada
def gerar_resposta_otimizada(pergunta, max_length=100, temperature=0.7):
    """
    Função otimizada para produção
    """
    try:
        # Formatando entrada
        input_text = f"User: {pergunta}\nAssistant:"
        
        # Tokenizando
        inputs = tokenizer(input_text, return_tensors="pt", truncation=True, max_length=512)
        
        # Gerando resposta
        with torch.no_grad():
            outputs = model.generate(
                inputs["input_ids"],
                max_length=max_length,
                num_return_sequences=1,
                temperature=temperature,
                do_sample=True,
                pad_token_id=tokenizer.eos_token_id
            )
        
        # Decodificando resposta
        resposta_completa = tokenizer.decode(outputs[0], skip_special_tokens=True)
        
        # Extraindo apenas a parte do assistant
        if "Assistant:" in resposta_completa:
            resposta = resposta_completa.split("Assistant:")[-1].strip()
        else:
            resposta = resposta_completa
        
        return resposta
        
    except Exception as e:
        return f"Erro na geração: {e}"

# Testando função otimizada
pergunta_teste = "Qual é o melhor iPhone?"
resposta = gerar_resposta_otimizada(pergunta_teste)

print(f"🧪 Teste de inferência:")
print(f"   Pergunta: {pergunta_teste}")
print(f"   Resposta: {resposta}")

print("\n✅ Modelo preparado para deploy!")

## **2. Deploy com Gradio (Interface Web)**

Vamos criar uma interface web simples:

In [ ]:
# Criando interface Gradio
print("🌐 CRIANDO INTERFACE GRADIO\n")

def interface_gradio(pergunta, max_length, temperature):
    """
    Interface Gradio para o modelo
    """
    if not pergunta.strip():
        return "Por favor, digite uma pergunta."
    
    resposta = gerar_resposta_otimizada(pergunta, max_length, temperature)
    return resposta

# Configurando interface
iface = gr.Interface(
    fn=interface_gradio,
    inputs=[
        gr.Textbox(label="❓ Faça sua pergunta sobre iPhone:", placeholder="Ex: Qual é o melhor iPhone para fotografia?"),
        gr.Slider(minimum=50, maximum=200, value=100, step=10, label="📏 Comprimento máximo da resposta"),
        gr.Slider(minimum=0.1, maximum=1.0, value=0.7, step=0.1, label="🌡️ Temperatura (criatividade)")
    ],
    outputs=gr.Textbox(label="🤖 Resposta da IA:", lines=5),
    title="📱 Assistente de Vendas iPhone",
    description="IA especializada em vendas de iPhone. Faça perguntas sobre produtos, preços e características!",
    examples=[
        ["Qual é o melhor iPhone para fotografia?", 100, 0.7],
        ["Quanto custa o iPhone 15?", 100, 0.7],
        ["Qual é a diferença entre iPhone 15 e iPhone 15 Pro?", 150, 0.7],
        ["O iPhone 15 tem carregamento sem fio?", 100, 0.7]
    ],
    theme=gr.themes.Soft()
)

print("✅ Interface Gradio criada!")
print("\n🚀 Para lançar a interface, execute:")
print("   iface.launch(share=True)")

# Salvando interface
iface.save("deploy_gradio")
print("✅ Interface salva em 'deploy_gradio'")

print("\n💡 Gradio cria uma interface web bonita e funcional automaticamente!")

## **3. Deploy com Streamlit (Aplicação Web)**

Vamos criar uma aplicação mais robusta:

In [ ]:
# Criando aplicação Streamlit
print("📱 CRIANDO APLICAÇÃO STREAMLIT\n")

streamlit_code = """
import streamlit as st
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
import time

# Configuração da página
st.set_page_config(
    page_title="📱 Assistente iPhone",
    page_icon="📱",
    layout="wide"
)

# Título
st.title("📱 Assistente de Vendas iPhone")
st.markdown("---")

# Sidebar com configurações
with st.sidebar:
    st.header("⚙️ Configurações")
    max_length = st.slider("📏 Comprimento máximo", 50, 200, 100)
    temperature = st.slider("🌡️ Temperatura", 0.1, 1.0, 0.7)
    
    st.markdown("---")
    st.markdown("### 📊 Estatísticas")
    st.metric("Perguntas respondidas", "0")
    st.metric("Tempo médio", "0.5s")

# Área principal
col1, col2 = st.columns([2, 1])

with col1:
    st.subheader("❓ Faça sua pergunta")
    pergunta = st.text_area(
        "Digite sua pergunta sobre iPhone:",
        placeholder="Ex: Qual é o melhor iPhone para fotografia?",
        height=100
    )
    
    if st.button("🚀 Gerar Resposta", type="primary"):
        if pergunta.strip():
            with st.spinner("🤖 IA pensando..."):
                start_time = time.time()
                # Aqui você chamaria sua função de geração
                resposta = "Esta é uma resposta simulada da IA treinada."
                end_time = time.time()
                
            st.success(f"✅ Resposta gerada em {end_time - start_time:.2f}s")
            st.markdown("### 🤖 Resposta:")
            st.write(resposta)
        else:
            st.error("Por favor, digite uma pergunta.")

with col2:
    st.subheader("💡 Exemplos")
    exemplos = [
        "Qual é o melhor iPhone para fotografia?",
        "Quanto custa o iPhone 15?",
        "Qual é a diferença entre iPhone 15 e iPhone 15 Pro?",
        "O iPhone 15 tem carregamento sem fio?"
    ]
    
    for exemplo in exemplos:
        if st.button(exemplo, key=exemplo):
            st.session_state.pergunta = exemplo
            st.rerun()

# Footer
st.markdown("---")
st.markdown("*Desenvolvido com ❤️ usando Fine Tuning*")
"""

# Salvando aplicação Streamlit
with open("app_streamlit.py", "w", encoding="utf-8") as f:
    f.write(streamlit_code)

print("✅ Aplicação Streamlit criada em 'app_streamlit.py'")
print("\n🚀 Para executar a aplicação:")
print("   streamlit run app_streamlit.py")

print("\n💡 Streamlit cria aplicações web mais robustas e interativas!")

## **4. Deploy em Hugging Face Spaces**

Vamos preparar para deploy na nuvem:

In [ ]:
# Preparando para Hugging Face Spaces
print("☁️ PREPARANDO PARA HUGGING FACE SPACES\n")

# Criando arquivo requirements.txt
requirements_content = """
torch>=2.0.0
transformers>=4.30.0
gradio>=3.40.0
streamlit>=1.25.0
numpy>=1.24.0
pandas>=2.0.0
"""

with open("requirements.txt", "w") as f:
    f.write(requirements_content)

print("✅ requirements.txt criado")

# Criando README para o Space
readme_content = """
# 📱 Assistente de Vendas iPhone

Este é um assistente de IA especializado em vendas de iPhone, treinado com Fine Tuning.

## 🚀 Como usar

1. Digite sua pergunta sobre iPhone
2. Ajuste os parâmetros se necessário
3. Clique em "Gerar Resposta"

## 💡 Exemplos de perguntas

- Qual é o melhor iPhone para fotografia?
- Quanto custa o iPhone 15?
- Qual é a diferença entre iPhone 15 e iPhone 15 Pro?
- O iPhone 15 tem carregamento sem fio?

## 🛠️ Tecnologias

- Fine Tuning com LoRA
- Transformers (Hugging Face)
- Gradio para interface

---

*Desenvolvido como parte do curso de Fine Tuning*
"""

with open("README.md", "w", encoding="utf-8") as f:
    f.write(readme_content)

print("✅ README.md criado")

# Criando app.py para Hugging Face Spaces
app_content = """
import gradio as gr
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

# Carregando modelo
model_name = "microsoft/DialoGPT-small"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)

if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

def gerar_resposta(pergunta, max_length, temperature):
    try:
        input_text = f"User: {pergunta}\nAssistant:"
        inputs = tokenizer(input_text, return_tensors="pt", truncation=True, max_length=512)
        
        with torch.no_grad():
            outputs = model.generate(
                inputs["input_ids"],
                max_length=max_length,
                num_return_sequences=1,
                temperature=temperature,
                do_sample=True,
                pad_token_id=tokenizer.eos_token_id
            )
        
        resposta_completa = tokenizer.decode(outputs[0], skip_special_tokens=True)
        
        if "Assistant:" in resposta_completa:
            resposta = resposta_completa.split("Assistant:")[-1].strip()
        else:
            resposta = resposta_completa
        
        return resposta
        
    except Exception as e:
        return f"Erro na geração: {e}"

# Interface Gradio
iface = gr.Interface(
    fn=gerar_resposta,
    inputs=[
        gr.Textbox(label="❓ Faça sua pergunta sobre iPhone:", placeholder="Ex: Qual é o melhor iPhone para fotografia?"),
        gr.Slider(minimum=50, maximum=200, value=100, step=10, label="📏 Comprimento máximo da resposta"),
        gr.Slider(minimum=0.1, maximum=1.0, value=0.7, step=0.1, label="🌡️ Temperatura (criatividade)")
    ],
    outputs=gr.Textbox(label="🤖 Resposta da IA:", lines=5),
    title="📱 Assistente de Vendas iPhone",
    description="IA especializada em vendas de iPhone. Faça perguntas sobre produtos, preços e características!",
    examples=[
        ["Qual é o melhor iPhone para fotografia?", 100, 0.7],
        ["Quanto custa o iPhone 15?", 100, 0.7],
        ["Qual é a diferença entre iPhone 15 e iPhone 15 Pro?", 150, 0.7],
        ["O iPhone 15 tem carregamento sem fio?", 100, 0.7]
    ],
    theme=gr.themes.Soft()
)

iface.launch()
"""

with open("app.py", "w", encoding="utf-8") as f:
    f.write(app_content)

print("✅ app.py criado")

print("\n🚀 Para fazer deploy no Hugging Face Spaces:")
print("   1. Crie um novo Space no Hugging Face")
print("   2. Faça upload dos arquivos: app.py, requirements.txt, README.md")
print("   3. O Space será automaticamente deployado!")

print("\n💡 Hugging Face Spaces oferece hospedagem gratuita para aplicações de IA!")

## **5. Deploy com API REST**

Vamos criar uma API para integração:

In [ ]:
# Criando API REST com FastAPI
print("🔌 CRIANDO API REST\n")

api_code = """
from fastapi import FastAPI, HTTPException
from pydantic import BaseModel
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
import uvicorn
from typing import Optional

# Configurando FastAPI
app = FastAPI(
    title="📱 Assistente iPhone API",
    description="API para assistente de vendas de iPhone treinado com Fine Tuning",
    version="1.0.0"
)

# Modelo de dados
class PerguntaRequest(BaseModel):
    pergunta: str
    max_length: Optional[int] = 100
    temperature: Optional[float] = 0.7

class RespostaResponse(BaseModel):
    pergunta: str
    resposta: str
    tempo_geracao: float
    status: str

# Carregando modelo (global)
model_name = "microsoft/DialoGPT-small"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)

if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

def gerar_resposta_api(pergunta, max_length, temperature):
    try:
        input_text = f"User: {pergunta}\nAssistant:"
        inputs = tokenizer(input_text, return_tensors="pt", truncation=True, max_length=512)
        
        with torch.no_grad():
            outputs = model.generate(
                inputs["input_ids"],
                max_length=max_length,
                num_return_sequences=1,
                temperature=temperature,
                do_sample=True,
                pad_token_id=tokenizer.eos_token_id
            )
        
        resposta_completa = tokenizer.decode(outputs[0], skip_special_tokens=True)
        
        if "Assistant:" in resposta_completa:
            resposta = resposta_completa.split("Assistant:")[-1].strip()
        else:
            resposta = resposta_completa
        
        return resposta
        
    except Exception as e:
        raise HTTPException(status_code=500, detail=f"Erro na geração: {e}")

@app.get("/")
async def root():
    return {"message": "📱 Assistente iPhone API - Funcionando!"}

@app.get("/health")
async def health_check():
    return {"status": "healthy", "model": model_name}

@app.post("/gerar-resposta", response_model=RespostaResponse)
async def gerar_resposta_endpoint(request: PerguntaRequest):
    import time
    
    if not request.pergunta.strip():
        raise HTTPException(status_code=400, detail="Pergunta não pode estar vazia")
    
    start_time = time.time()
    resposta = gerar_resposta_api(request.pergunta, request.max_length, request.temperature)
    end_time = time.time()
    
    return RespostaResponse(
        pergunta=request.pergunta,
        resposta=resposta,
        tempo_geracao=end_time - start_time,
        status="success"
    )

@app.get("/exemplos")
async def get_exemplos():
    return {
        "exemplos": [
            "Qual é o melhor iPhone para fotografia?",
            "Quanto custa o iPhone 15?",
            "Qual é a diferença entre iPhone 15 e iPhone 15 Pro?",
            "O iPhone 15 tem carregamento sem fio?"
        ]
    }

if __name__ == "__main__":
    uvicorn.run(app, host="0.0.0.0", port=8000)
"""

# Salvando API
with open("api_fastapi.py", "w", encoding="utf-8") as f:
    f.write(api_code)

print("✅ API FastAPI criada em 'api_fastapi.py'")
print("\n🚀 Para executar a API:")
print("   python api_fastapi.py")
print("   # A API estará disponível em http://localhost:8000")

# Criando exemplo de uso da API
exemplo_uso_api = """
import requests

# Exemplo de uso da API
url = "http://localhost:8000/gerar-resposta"

data = {
    "pergunta": "Qual é o melhor iPhone para fotografia?",
    "max_length": 100,
    "temperature": 0.7
}

response = requests.post(url, json=data)
resultado = response.json()

print(f"Pergunta: {resultado['pergunta']}")
print(f"Resposta: {resultado['resposta']}")
print(f"Tempo: {resultado['tempo_geracao']:.2f}s")
"""

with open("exemplo_uso_api.py", "w", encoding="utf-8") as f:
    f.write(exemplo_uso_api)

print("✅ Exemplo de uso da API criado")

print("\n💡 API REST permite integração com qualquer aplicação!")
print("   - Web apps, mobile apps, chatbots, etc.")

## **6. Comparação de métodos de deploy**

Vamos comparar as diferentes opções:

In [ ]:
# Comparação de métodos de deploy
print("📊 COMPARAÇÃO DE MÉTODOS DE DEPLOY\n")

metodos_deploy = {
    "Gradio": {
        "facilidade": "Muito fácil",
        "velocidade": "Rápido",
        "customizacao": "Limitada",
        "custo": "Gratuito",
        "melhor_para": "Prototipagem, demonstrações",
        "limitacoes": "Interface básica"
    },
    "Streamlit": {
        "facilidade": "Fácil",
        "velocidade": "Rápido",
        "customizacao": "Média",
        "custo": "Gratuito",
        "melhor_para": "Dashboards, aplicações web",
        "limitacoes": "Menos flexível que frameworks web"
    },
    "Hugging Face Spaces": {
        "facilidade": "Muito fácil",
        "velocidade": "Muito rápido",
        "customizacao": "Média",
        "custo": "Gratuito",
        "melhor_para": "Compartilhamento, demonstrações",
        "limitacoes": "Recursos limitados"
    },
    "API REST": {
        "facilidade": "Médio",
        "velocidade": "Rápido",
        "customizacao": "Alta",
        "custo": "Variável",
        "melhor_para": "Integração, aplicações empresariais",
        "limitacoes": "Precisa de infraestrutura"
    }
}

for metodo, info in metodos_deploy.items():
    print(f"🔹 {metodo}")
    print(f"   🎯 Facilidade: {info['facilidade']}")
    print(f"   ⚡ Velocidade: {info['velocidade']}")
    print(f"   🎨 Customização: {info['customizacao']}")
    print(f"   💰 Custo: {info['custo']}")
    print(f"   🎯 Melhor para: {info['melhor_para']}")
    print(f"   ⚠️  Limitações: {info['limitacoes']}")
    print()

print("💡 Escolha o método baseado no seu caso de uso e recursos disponíveis!")

## **7. Checklist de deploy**

Vamos criar um checklist para garantir um deploy bem-sucedido:

In [ ]:
# Checklist de deploy
print("✅ CHECKLIST DE DEPLOY\n")

checklist = [
    {
        "categoria": "📋 Preparação",
        "itens": [
            "Modelo treinado e testado",
            "Dados de exemplo preparados",
            "Função de inferência otimizada",
            "Tratamento de erros implementado"
        ]
    },
    {
        "categoria": "🌐 Interface",
        "itens": [
            "Interface de usuário criada",
            "Exemplos de uso incluídos",
            "Validação de entrada implementada",
            "Feedback visual para o usuário"
        ]
    },
    {
        "categoria": "🚀 Deploy",
        "itens": [
            "Arquivos de configuração criados",
            "Dependências listadas",
            "Documentação incluída",
            "Testes de funcionamento realizados"
        ]
    },
    {
        "categoria": "📊 Monitoramento",
        "itens": [
            "Logs de erro configurados",
            "Métricas de performance definidas",
            "Sistema de backup implementado",
            "Plano de escalabilidade definido"
        ]
    }
]

for categoria in checklist:
    print(f"{categoria['categoria']}:")
    for item in categoria['itens']:
        print(f"   ☐ {item}")
    print()

print("💡 Marque cada item conforme for implementando!")
print("🎯 Um deploy bem planejado evita problemas futuros!")

## **8. Teste Rápido**

Vamos testar seu entendimento sobre deploy:

In [ ]:
# Teste rápido sobre deploy
print("🧪 TESTE RÁPIDO - DEPLOY\n")

perguntas_teste = [
    {
        "pergunta": "Qual é o método de deploy mais fácil para prototipagem?",
        "opcoes": [
            "A) API REST", 
            "B) Gradio", 
            "C) Streamlit"
        ],
        "resposta": "B",
        "explicacao": "Gradio é o mais fácil para prototipagem rápida!"
    },
    {
        "pergunta": "Qual plataforma oferece hospedagem gratuita para aplicações de IA?",
        "opcoes": [
            "A) AWS", 
            "B) Hugging Face Spaces", 
            "C) Google Cloud"
        ],
        "resposta": "B",
        "explicacao": "Hugging Face Spaces oferece hospedagem gratuita!"
    },
    {
        "pergunta": "Qual método é melhor para integração com outras aplicações?",
        "opcoes": [
            "A) Gradio", 
            "B) API REST", 
            "C) Streamlit"
        ],
        "resposta": "B",
        "explicacao": "API REST permite integração com qualquer aplicação!"
    }
]

for i, q in enumerate(perguntas_teste, 1):
    print(f"❓ {i}. {q['pergunta']}")
    for opcao in q['opcoes']:
        print(f"   {opcao}")
    print(f"💡 Resposta: {q['resposta']} - {q['explicacao']}")
    print()

print("🎉 Parabéns! Você já entende os fundamentos do deploy de modelos!")

## **🎉 Módulo 6 Concluído!**

### **O que aprendemos:**

✅ **Preparação de modelo** para deploy  
✅ **Deploy com Gradio** (interface web simples)  
✅ **Deploy com Streamlit** (aplicação web robusta)  
✅ **Deploy em Hugging Face Spaces** (hospedagem gratuita)  
✅ **Deploy com API REST** (integração)  
✅ **Comparação de métodos** de deploy  
✅ **Checklist de deploy** completo

### **🎓 Curso Completo!**

🚀 **Módulo 1**: Introdução ao Fine Tuning  
🚀 **Módulo 2**: Preparação de dados  
🚀 **Módulo 3**: Escolha de modelos  
🚀 **Módulo 4**: Treinamento  
🚀 **Módulo 5**: Avaliação  
🚀 **Módulo 6**: Deploy ✅

---

**💡 Dica do Instrutor**: Deploy é como abrir um restaurante - você precisa de uma boa localização (hospedagem), um cardápio atrativo (interface) e um serviço de qualidade (modelo bem treinado)! Agora sua IA está pronta para atender clientes reais! 😄

**🎉 Parabéns! Você completou o curso de Fine Tuning!**

**🚀 Agora você é um treinador de IA!**